# Check Number of FEL trains (single bunch) in TimePix HDF5 files

In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


#### Imports

In [2]:
import numpy as np  
import matplotlib.pyplot as plt
import h5py
import os

#### Utility functions

In [82]:
def generate_hdf_file_name(run_number):
    try:
        hdf_file_path = '/gpfs/bl1/current/processed/timepix_hdf/'
        file_start = "run_"+str(run_number).zfill(4)
        not_file_end = 'rawOnly.hdf5'
        hdf_file = [i for i in os.listdir(hdf_file_path) if os.path.isfile(os.path.join(hdf_file_path,i)) and i.startswith(file_start) and not i.endswith(not_file_end)][0]
        hdf_file_complete_path = hdf_file_path+hdf_file
        assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'
        return hdf_file_complete_path
    except IndexError:
        print("Run",run_number,"do not exist!")

def get_train_numbers_from_file(run_interval):
    number_of_trains = []
    runs = []
    for i in range(run_interval[0], run_interval[1]+1):
        runs.append(i)
        hdf_file = generate_hdf_file_name(i)
        if hdf_file != None:
            number_of_trains.append(number_of_trains_from_hdf(hdf_file))
        elif hdf_file == None:
            number_of_trains.append(0)
    return runs, number_of_trains

def number_of_trains_from_hdf(hdf_file_complete_path):
    '''Retrun number of recorded FEL trains in HDF file'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        trains = len(h_file['tpx3Times/triggerNr'][:])
    return trains

def unique(list_to_check):  
    unique_list = [] 
    for x in list_to_check: 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

def get_trains_vs_unique_delay(delay_pos, trains):
    unique_delay_pos = unique(delay_pos)
    unique_trains = []
    for i in range(len(unique_delay_pos)):
        tt = []
        for j in range(len(delay_pos)):
            if unique_delay_pos[i] == delay_pos[j]:
                tt.append(trains[j])
        unique_trains.append(np.sum(np.array(tt)))
    sort_ind = np.argsort(np.array(unique_delay_pos))
    unique_delay_pos = np.array(unique_delay_pos)[sort_ind]
    unique_trains = np.array(unique_trains)[sort_ind]
    return unique_delay_pos, unique_trains

def plot_number_of_trains_vs_run(run_interval): 
    runs, trains = get_train_numbers_from_file(run_interval)              
    fig = plt.figure(figsize = (10,10))
    plt.plot(runs,trains,'o')
    plt.ylabel('FEL pulses')
    plt.xlabel('Run Number')
    plt.show()
        
def list_number_of_trains_vs_run(run_interval): 
    runs, trains = get_train_numbers_from_file(run_interval)
    for i in range(len(runs)):
        print('RunNumber:',runs[i],'| number of trains:',trains[i])

def plot_number_of_trains_vs_delay(run_interval):
    runs, trains = get_train_numbers_from_file(run_interval)
    runs_pp, delay_pos = get_delay_stage_pos_from_txt_file(run_interval)
    assert runs == runs_pp, 'Runs are not identical!'
    unique_delay_pos, unique_trains = get_trains_vs_unique_delay(delay_pos, trains)
    fig = plt.figure(figsize = (10,10))
    plt.plot(unique_delay_pos,unique_trains,'o')
    plt.ylabel('FEL pulses')
    plt.xlabel('Delay Position')
    plt.show()
        
def list_number_of_trains_vs_delay(run_interval):
    runs, trains = get_train_numbers_from_file(run_interval)
    runs_pp, delay_pos = get_delay_stage_pos_from_txt_file(run_interval)
    assert runs == runs_pp, 'Runs are not identical!'
    unique_delay_pos, unique_trains = get_trains_vs_unique_delay(delay_pos, trains)
    for i in range(len(unique_delay_pos)):
        print('Delay Position:', unique_delay_pos[i],'| number of trains:',unique_trains[i])           

def get_delay_stage_pos_from_txt_file(run_interval):
    runs = []
    delay_pos = []
    filepath = '/home/bl1user/Desktop/erk20919/CAMP/beamtime/run_pp-delay.txt'
    with open(filepath) as fp:
        for cnt, line in enumerate(fp):
            tt = [x.strip() for x in line.split(',')]
            runs.append(int(tt[0]))
            delay_pos.append(float(tt[1]))
    delay_pos = delay_pos[runs.index(run_interval[0]):runs.index(run_interval[1])+1]
    runs = runs[runs.index(run_interval[0]):runs.index(run_interval[1])+1]
    return runs, delay_pos


### Interval of runs

In [104]:
run_interval = [185, 211]

### Number of trains in runs

In [105]:
# plot_number_of_trains_vs_run(run_interval)
list_number_of_trains_vs_run(run_interval)

Run 211 do not exist!
RunNumber: 185 | number of trains: 5703
RunNumber: 186 | number of trains: 6253
RunNumber: 187 | number of trains: 6038
RunNumber: 188 | number of trains: 6053
RunNumber: 189 | number of trains: 6541
RunNumber: 190 | number of trains: 6542
RunNumber: 191 | number of trains: 5985
RunNumber: 192 | number of trains: 6083
RunNumber: 193 | number of trains: 6275
RunNumber: 194 | number of trains: 5803
RunNumber: 195 | number of trains: 6312
RunNumber: 196 | number of trains: 5999
RunNumber: 197 | number of trains: 5875
RunNumber: 198 | number of trains: 5985
RunNumber: 199 | number of trains: 6060
RunNumber: 200 | number of trains: 6809
RunNumber: 201 | number of trains: 6181
RunNumber: 202 | number of trains: 6155
RunNumber: 203 | number of trains: 6194
RunNumber: 204 | number of trains: 6215
RunNumber: 205 | number of trains: 6637
RunNumber: 206 | number of trains: 5016
RunNumber: 207 | number of trains: 5014
RunNumber: 208 | number of trains: 6377
RunNumber: 209 | n

### Number of trains in delay steps

In [98]:
# plot_number_of_trains_vs_delay(run_interval)
list_number_of_trains_vs_delay(run_interval)

Delay Position: 330.0 | number of trains: 6181
Delay Position: 335.0 | number of trains: 6155
Delay Position: 340.0 | number of trains: 6809
Delay Position: 341.0 | number of trains: 6312
Delay Position: 342.0 | number of trains: 6541
Delay Position: 343.0 | number of trains: 6053
Delay Position: 344.0 | number of trains: 6038
Delay Position: 344.25 | number of trains: 6215
Delay Position: 344.5 | number of trains: 5985
Delay Position: 344.75 | number of trains: 6194
Delay Position: 345.0 | number of trains: 11763
Delay Position: 345.25 | number of trains: 5803
Delay Position: 345.5 | number of trains: 6083
Delay Position: 345.75 | number of trains: 6275
Delay Position: 345.82 | number of trains: 785
Delay Position: 346.0 | number of trains: 6253
Delay Position: 346.25 | number of trains: 5656
Delay Position: 346.5 | number of trains: 5999
Delay Position: 346.75 | number of trains: 3787
Delay Position: 347.0 | number of trains: 6542
Delay Position: 347.5 | number of trains: 5875
Delay 